# **Final_v1**

In [1]:
import requests
import pandas as pd

# Function to construct the API request URL
def construct_link(table_id, variables):
    base = f'https://api.statbank.dk/v1/data/{table_id}/JSONSTAT?lang=en'
    for var in variables:
        base += f'&{var}'
    return base

# Function to get weighted average age per municipality
def get_weighted_average_age():
    age_groups = [str(i) for i in range(101)]
    age_weighted_sums = {}
    total_population = {}
    
    # Initialize dictionaries for each municipality
    url_initial = construct_link('FOLK1A', [f'Tid=2023K1', 'OMRÅDE=*', 'KØN=TOT', f'ALDER=0'])
    response_initial = requests.get(url_initial)
    if response_initial.status_code == 200:
        data = response_initial.json()
        municipalities = data['dataset']['dimension']['OMRÅDE']['category']['label'].values()
        for municipality in municipalities:
            age_weighted_sums[municipality] = 0
            total_population[municipality] = 0

    # Process each age group
    for age in age_groups:
        url = construct_link('FOLK1A', [f'Tid=2023K1', 'OMRÅDE=*', 'KØN=TOT', f'ALDER={age}'])
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            values = data['dataset']['value']
            municipalities = data['dataset']['dimension']['OMRÅDE']['category']['label'].values()
            for i, municipality in enumerate(municipalities):
                age_weighted_sums[municipality] += int(age) * values[i]
                total_population[municipality] += values[i]
    
    # Calculate the average age for each municipality
    average_age_data = []
    for municipality in municipalities:
        if total_population[municipality] > 0:
            avg_age = age_weighted_sums[municipality] / total_population[municipality]
            average_age_data.append((municipality, avg_age))
        else:
            average_age_data.append((municipality, None))

    df_average_age = pd.DataFrame(average_age_data, columns=["Municipality", "AverageAge"])
    return df_average_age

# Function to get number of children under 18 per municipality
def get_number_of_children():
    age_groups = [str(i) for i in range(18)]
    children_data = []

    for age in age_groups:
        url = construct_link('FOLK1A', [f'Tid=2023K1', 'OMRÅDE=*', 'KØN=TOT', f'ALDER={age}'])
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            values = data['dataset']['value']
            if not children_data:
                children_data = [0] * len(values)
            children_data = [children_data[i] + values[i] for i in range(len(values))]

    municipalities = data['dataset']['dimension']['OMRÅDE']['category']['label'].values()
    df_children = pd.DataFrame(list(zip(municipalities, children_data)), columns=["Municipality", "NumberOfChildren"])
    return df_children

# Function to get population by gender per municipality
def get_population_by_gender():
    url_men = construct_link('FOLK1A', ['Tid=2023K1', 'OMRÅDE=*', 'KØN=1', 'ALDER=IALT'])
    url_women = construct_link('FOLK1A', ['Tid=2023K1', 'OMRÅDE=*', 'KØN=2', 'ALDER=IALT'])

    response_men = requests.get(url_men)
    response_women = requests.get(url_women)
    
    if response_men.status_code == 200 and response_women.status_code == 200:
        data_men = response_men.json()
        data_women = response_women.json()
        values_men = data_men['dataset']['value']
        values_women = data_women['dataset']['value']
        municipalities = data_men['dataset']['dimension']['OMRÅDE']['category']['label'].values()

        df_gender = pd.DataFrame(list(zip(municipalities, values_men, values_women)), columns=["Municipality", "Men", "Women"])
        return df_gender
    else:
        return pd.DataFrame(columns=["Municipality", "Men", "Women"])

# Function to get average income per municipality
def get_average_income():
    url = construct_link('INDKP105', ['Tid=2022', 'OMRÅDE=*', 'ENHED=118', 'KOEN=MOK', 'ALDER1=00', 'INDKINTB=000'])
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        values = data['dataset']['value']
        municipalities = data['dataset']['dimension']['OMRÅDE']['category']['label'].values()
        df_income = pd.DataFrame(list(zip(municipalities, values)), columns=["Municipality", "AverageIncome"])
        return df_income
    else:
        return pd.DataFrame(columns=["Municipality", "AverageIncome"])

# Function to get education data per level and municipality
def get_education_data_by_level(education_level):
    url = construct_link('UDDAKT10', ['Tid=2022', 'BOPOMR=*', f'UDDANNELSE={education_level}', 'ALDER=TOT', 'KØN=10', 'FSTATUS=B'])
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        values = data['dataset']['value']
        municipalities = data['dataset']['dimension']['BOPOMR']['category']['label'].values()
        df = pd.DataFrame(list(zip(municipalities, values)), columns=["Municipality", education_level])
        return df
    else:
        return pd.DataFrame(columns=["Municipality", education_level])

# Combine demographic and income data into one dataset
def combine_data():
    df_age = get_weighted_average_age()
    df_children = get_number_of_children()
    df_gender = get_population_by_gender()
    df_income = get_average_income()

    # Merge all datasets on Municipality column
    df_combined = df_age.merge(df_children, on="Municipality", how='outer').merge(df_gender, on="Municipality", how='outer').merge(df_income, on="Municipality", how="outer")
    
    return df_combined

# Get all education levels
education_levels = ["H10", "H20", "H30", "H40", "H50", "H60", "H70", "H80"]

# Retrieve data for all education levels and merge them into a single DataFrame
df_education_combined = None
for level in education_levels:
    df_level = get_education_data_by_level(level)
    if df_education_combined is None:
        df_education_combined = df_level
    else:
        df_education_combined = pd.merge(df_education_combined, df_level, on="Municipality", how="outer")

# Combine demographic and income data with education data
df_combined = combine_data()  # First combine demographic and income data
df_final_combined = pd.merge(df_combined, df_education_combined, on="Municipality", how="left")  # Then merge education data

# Display the final combined dataset
print(df_final_combined.head())

# Optionally save the final combined dataset to a CSV
df_final_combined.to_csv("final_combined_dataset.csv", index=False)

# Example lookup for a specific municipality (e.g., Copenhagen)
def lookup_final_municipality_data(df, municipality_name):
    df_municipality = df[df["Municipality"] == municipality_name]
    if df_municipality.empty:
        print(f"No data found for municipality: {municipality_name}")
    else:
        print(f"Data for {municipality_name}:")
        print(df_municipality)

# Lookup example for 'Copenhagen'
lookup_final_municipality_data(df_final_combined, 'Copenhagen')


  Municipality  AverageAge  NumberOfChildren        Men      Women  \
0     Aabenraa   44.442998           11416.0    29422.0    29580.0   
1      Aalborg   39.787919           40001.0   111992.0   110579.0   
2       Aarhus   37.468004           64420.0   177450.0   184094.0   
3  Albertslund   40.197494            5886.0    13669.0    13861.0   
4  All Denmark   41.699473         1155838.0  2948999.0  2983655.0   

   AverageIncome       H10       H20       H30      H40      H50      H60  \
0       337605.0    6870.0    1357.0    1076.0     70.0    573.0     33.0   
1       344376.0   22874.0    4884.0    4571.0   1678.0   6219.0   7230.0   
2       361364.0   36370.0    7686.0    5184.0   3064.0  15269.0  14294.0   
3       322050.0    3535.0     726.0     617.0    172.0    600.0    343.0   
4       371900.0  681384.0  141864.0  100202.0  23937.0  93519.0  73475.0   

       H70     H80  
0     49.0     8.0  
1   4844.0   582.0  
2  10542.0  1470.0  
3    336.0    17.0  
4  62355.0 

In [2]:
df_final_combined

,Municipality,AverageAge,NumberOfChildren,Men,Women,AverageIncome,H10,H20,H30,H40,H50,H60,H70,H80
0,Aabenraa,44.442998,11416.0,29422.0,29580.0,337605.0,6870.0,1357.0,1076.0,70.0,573.0,33.0,49.0,8.0
1,Aalborg,39.787919,40001.0,111992.0,110579.0,344376.0,22874.0,4884.0,4571.0,1678.0,6219.0,7230.0,4844.0,582.0
2,Aarhus,37.468004,64420.0,177450.0,184094.0,361364.0,36370.0,7686.0,5184.0,3064.0,15269.0,14294.0,10542.0,1470.0
3,Albertslund,40.197494,5886.0,13669.0,13861.0,322050.0,3535.0,726.0,617.0,172.0,600.0,343.0,336.0,17.0
4,All Denmark,41.699473,1155838.0,2948999.0,2983655.0,371900.0,681384.0,141864.0,100202.0,23937.0,93519.0,73475.0,62355.0,9651.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Vejle,41.044520,25719.0,60522.0,60427.0,378172.0,15128.0,3102.0,2122.0,437.0,1360.0,176.0,237.0,55.0
112,Vesthimmerlands,44.592912,7078.0,18402.0,18029.0,339304.0,4379.0,672.0,748.0,34.0,163.0,35.0,26.0,4.0
113,Viborg,42.297720,20466.0,48994.0,48737.0,354679.0,12556.0,2565.0,2118.0,267.0,1055.0,90.0,116.0,53.0
114,Vordingborg,47.128792,7526.0,22700.0,22741.0,334910.0,4541.0,942.0,699.0,99.0,473.0,41.0,51.0,7.0


## Dataset for socioeconomic data ##

In [4]:
import requests
import pandas as pd

# Define the categories to extract related to real estate impact, along with their descriptions
relevant_functions = {
    "32201": "Folkeskoler",  # Folkeskoler
    "32207": "Special Education",  # Special Education
    "53027": "Elderly Care",  # Elderly Care
    "46288": "Healthcare",  # Healthcare (Sundhedsfremme og forebyggelse)
    "03853": "Green Spaces",  # Green Spaces (Skove)
    "22811": "Infrastructure",  # Infrastructure (Vejvedligeholdelse)
    "53859": "Social Services"  # Social Services (Aktivitets- og samværstilbud)
}

# Function to construct the API link
def construct_link(table_id, variables):
    base = f'https://api.statbank.dk/v1/data/{table_id}/JSONSTAT?lang=en'
    for var in variables:
        base += f'&{var}'
    return base

# Function to retrieve municipal budget data for a specific function
def get_municipal_budget_data_by_function(function_code, description):
    url = construct_link('BUDK32', [f'Tid=2023', f'FUNKTION={function_code}', 'REGI07A=*', 'DRANST=1', 'ART=UE', 'PRISENHED=LOBM'])
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        values = data['dataset']['value']
        municipalities = data['dataset']['dimension']['REGI07A']['category']['label'].values()
        
        # Multiply the values by 1,000 and create a DataFrame with the proper column name
        df = pd.DataFrame(list(zip(municipalities, [v * 1000 for v in values])), columns=["Municipality", description])
        return df
    else:
        print(f"Error fetching data for function {description}: {response.status_code}")
        return pd.DataFrame(columns=["Municipality", description])

# Retrieve data for all relevant functions and merge them into a single DataFrame
df_budget_combined = None

for function_code, description in relevant_functions.items():
    df_function = get_municipal_budget_data_by_function(function_code, description)
    if df_budget_combined is None:
        df_budget_combined = df_function
    else:
        df_budget_combined = pd.merge(df_budget_combined, df_function, on="Municipality", how="outer")

# Display the combined budget data for all municipalities and functions
df_budget_combined.head()

,Municipality,Folkeskoler,Special Education,Elderly Care,Healthcare,Green Spaces,Infrastructure,Social Services
0,Aabenraa,368550000,679000,322629000,1710000,28000,46206000,35892000
1,Aalborg,1620359000,5402000,1117856000,58139000,1610000,132129000,134689000
2,Aarhus,2226985000,4347000,1442222000,68896000,5633000,75722000,113920000
3,Albertslund,261231000,1201000,154417000,7708000,35000,26054000,30108000
4,All Denmark,40237510000,185958000,30797574000,1427778000,77979000,3265476000,4115113000


In [5]:
import requests
import pandas as pd

# Define the education level mappings
education_level_names = {
    "H10": "Elementary school",
    "H20": "High school education",
    "H30": "Vocational Education",
    "H40": "Short Higher Education",
    "H50": "Intermediate Higher Education",
    "H60": "Bachelor's programmes",
    "H70": "Longer Higher Education",
    "H80": "PhD and research programs"
}

# Define the categories to extract related to real estate impact, along with their descriptions
relevant_functions = {
    "32201": "Elementary school expense",
    "32207": "Special Education expense",
    "53027": "Elderly Care expense",
    "46288": "Healthcare expense",
    "03853": "Green Spaces expense",
    "22811": "Infrastructure expense",
    "53859": "Social Services expense"
}

# Function to construct the API request URL
def construct_link(table_id, variables):
    base = f'https://api.statbank.dk/v1/data/{table_id}/JSONSTAT?lang=en'
    for var in variables:
        base += f'&{var}'
    return base

# Function to get weighted average age per municipality
def get_weighted_average_age():
    age_groups = [str(i) for i in range(101)]
    age_weighted_sums = {}
    total_population = {}
    
    # Initialize dictionaries for each municipality
    url_initial = construct_link('FOLK1A', [f'Tid=2023K1', 'OMRÅDE=*', 'KØN=TOT', f'ALDER=0'])
    response_initial = requests.get(url_initial)
    if response_initial.status_code == 200:
        data = response_initial.json()
        municipalities = data['dataset']['dimension']['OMRÅDE']['category']['label'].values()
        for municipality in municipalities:
            age_weighted_sums[municipality] = 0
            total_population[municipality] = 0

    # Process each age group
    for age in age_groups:
        url = construct_link('FOLK1A', [f'Tid=2023K1', 'OMRÅDE=*', 'KØN=TOT', f'ALDER={age}'])
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            values = data['dataset']['value']
            municipalities = data['dataset']['dimension']['OMRÅDE']['category']['label'].values()
            for i, municipality in enumerate(municipalities):
                age_weighted_sums[municipality] += int(age) * values[i]
                total_population[municipality] += values[i]
    
    # Calculate the average age for each municipality
    average_age_data = []
    for municipality in municipalities:
        if total_population[municipality] > 0:
            avg_age = age_weighted_sums[municipality] / total_population[municipality]
            average_age_data.append((municipality, avg_age))
        else:
            average_age_data.append((municipality, None))

    df_average_age = pd.DataFrame(average_age_data, columns=["Municipality", "AverageAge"])
    return df_average_age

# Function to get number of children under 18 per municipality
def get_number_of_children():
    age_groups = [str(i) for i in range(18)]
    children_data = []

    for age in age_groups:
        url = construct_link('FOLK1A', [f'Tid=2023K1', 'OMRÅDE=*', 'KØN=TOT', f'ALDER={age}'])
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            values = data['dataset']['value']
            if not children_data:
                children_data = [0] * len(values)
            children_data = [children_data[i] + values[i] for i in range(len(values))]

    municipalities = data['dataset']['dimension']['OMRÅDE']['category']['label'].values()
    df_children = pd.DataFrame(list(zip(municipalities, children_data)), columns=["Municipality", "NumberOfChildren"])
    return df_children

# Function to get population by gender per municipality
def get_population_by_gender():
    url_men = construct_link('FOLK1A', ['Tid=2023K1', 'OMRÅDE=*', 'KØN=1', 'ALDER=IALT'])
    url_women = construct_link('FOLK1A', ['Tid=2023K1', 'OMRÅDE=*', 'KØN=2', 'ALDER=IALT'])

    response_men = requests.get(url_men)
    response_women = requests.get(url_women)
    
    if response_men.status_code == 200 and response_women.status_code == 200:
        data_men = response_men.json()
        data_women = response_women.json()
        values_men = data_men['dataset']['value']
        values_women = data_women['dataset']['value']
        municipalities = data_men['dataset']['dimension']['OMRÅDE']['category']['label'].values()

        df_gender = pd.DataFrame(list(zip(municipalities, values_men, values_women)), columns=["Municipality", "Men", "Women"])
        return df_gender
    else:
        return pd.DataFrame(columns=["Municipality", "Men", "Women"])

# Function to get average income per municipality
def get_average_income():
    url = construct_link('INDKP105', ['Tid=2022', 'OMRÅDE=*', 'ENHED=118', 'KOEN=MOK', 'ALDER1=00', 'INDKINTB=000'])
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        values = data['dataset']['value']
        municipalities = data['dataset']['dimension']['OMRÅDE']['category']['label'].values()
        df_income = pd.DataFrame(list(zip(municipalities, values)), columns=["Municipality", "AverageIncome"])
        return df_income
    else:
        return pd.DataFrame(columns=["Municipality", "AverageIncome"])

# Function to get education data per level and municipality
def get_education_data_by_level(education_level):
    url = construct_link('UDDAKT10', ['Tid=2022', 'BOPOMR=*', f'UDDANNELSE={education_level}', 'ALDER=TOT', 'KØN=10', 'FSTATUS=B'])
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        values = data['dataset']['value']
        municipalities = data['dataset']['dimension']['BOPOMR']['category']['label'].values()
        df = pd.DataFrame(list(zip(municipalities, values)), columns=["Municipality", education_level])
        return df
    else:
        return pd.DataFrame(columns=["Municipality", education_level])

# Function to get budget data per function
def get_municipal_budget_data_by_function(function_code, description):
    url = construct_link('BUDK32', [f'Tid=2023', f'FUNKTION={function_code}', 'REGI07A=*', 'DRANST=1', 'ART=UE', 'PRISENHED=LOBM'])
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        values = data['dataset']['value']
        municipalities = data['dataset']['dimension']['REGI07A']['category']['label'].values()
        
        # Multiply the values by 1,000 and create a DataFrame with the proper column name
        df = pd.DataFrame(list(zip(municipalities, [v * 1000 for v in values])), columns=["Municipality", description])
        return df
    else:
        print(f"Error fetching data for function {description}: {response.status_code}")
        return pd.DataFrame(columns=["Municipality", description])

# Combine demographic, income, education and budget data into one dataset
def combine_all_data():
    df_age = get_weighted_average_age()
    df_children = get_number_of_children()
    df_gender = get_population_by_gender()
    df_income = get_average_income()

    # Merge demographic and income datasets
    df_combined = df_age.merge(df_children, on="Municipality", how='outer')\
                        .merge(df_gender, on="Municipality", how='outer')\
                        .merge(df_income, on="Municipality", how="outer")
    
    # Get all education levels
    education_levels = ["H10", "H20", "H30", "H40", "H50", "H60", "H70", "H80"]
    df_education_combined = None
    for level in education_levels:
        df_level = get_education_data_by_level(level)
        if df_education_combined is None:
            df_education_combined = df_level
        else:
            df_education_combined = pd.merge(df_education_combined, df_level, on="Municipality", how="outer")

    # Rename the education columns to their proper names
    df_education_combined.rename(columns=education_level_names, inplace=True)

    # Merge education data into the combined dataset
    df_final_combined = pd.merge(df_combined, df_education_combined, on="Municipality", how="left")

    # Retrieve budget data for all relevant functions and merge them into the dataset
    df_budget_combined = None
    for function_code, description in relevant_functions.items():
        df_function = get_municipal_budget_data_by_function(function_code, description)
        if df_budget_combined is None:
            df_budget_combined = df_function
        else:
            df_budget_combined = pd.merge(df_budget_combined, df_function, on="Municipality", how="outer")

    # Merge the final combined data with the budget data
    df_final_combined = pd.merge(df_final_combined, df_budget_combined, on="Municipality", how="left")
    
    return df_final_combined

# Get the final combined data
df_final_data = combine_all_data()

# Display the final combined dataset
display(df_final_data.head())


,Municipality,AverageAge,NumberOfChildren,Men,Women,AverageIncome,Elementary school,High school education,Vocational Education,Short Higher Education,...,Bachelor's programmes,Longer Higher Education,PhD and research programs,Elementary school expense,Special Education expense,Elderly Care expense,Healthcare expense,Green Spaces expense,Infrastructure expense,Social Services expense
0,Aabenraa,44.442998,11416.0,29422.0,29580.0,337605.0,6870.0,1357.0,1076.0,70.0,...,33.0,49.0,8.0,3.685500e+08,679000.0,3.226290e+08,1.710000e+06,28000.0,4.620600e+07,3.589200e+07
1,Aalborg,39.787919,40001.0,111992.0,110579.0,344376.0,22874.0,4884.0,4571.0,1678.0,...,7230.0,4844.0,582.0,1.620359e+09,5402000.0,1.117856e+09,5.813900e+07,1610000.0,1.321290e+08,1.346890e+08
2,Aarhus,37.468004,64420.0,177450.0,184094.0,361364.0,36370.0,7686.0,5184.0,3064.0,...,14294.0,10542.0,1470.0,2.226985e+09,4347000.0,1.442222e+09,6.889600e+07,5633000.0,7.572200e+07,1.139200e+08
3,Albertslund,40.197494,5886.0,13669.0,13861.0,322050.0,3535.0,726.0,617.0,172.0,...,343.0,336.0,17.0,2.612310e+08,1201000.0,1.544170e+08,7.708000e+06,35000.0,2.605400e+07,3.010800e+07
4,All Denmark,41.699473,1155838.0,2948999.0,2983655.0,371900.0,681384.0,141864.0,100202.0,23937.0,...,73475.0,62355.0,9651.0,4.023751e+10,185958000.0,3.079757e+10,1.427778e+09,77979000.0,3.265476e+09,4.115113e+09


In [6]:
# Replace 'Copenhagen' with 'København' in the 'Municipality' column
df_final_data['Municipality'] = df_final_data['Municipality'].replace('Copenhagen', 'København')

# Check if the change was successful
display(df_final_data[df_final_data['Municipality'] == 'København'])


,Municipality,AverageAge,NumberOfChildren,Men,Women,AverageIncome,Elementary school,High school education,Vocational Education,Short Higher Education,...,Bachelor's programmes,Longer Higher Education,PhD and research programs,Elementary school expense,Special Education expense,Elderly Care expense,Healthcare expense,Green Spaces expense,Infrastructure expense,Social Services expense
13,København,35.955329,110835.0,321808.0,331856.0,383030.0,58089.0,11631.0,7878.0,4449.0,...,24159.0,22620.0,3205.0,3.263755e+09,25776000.0,2.885633e+09,303507000.0,0.0,342198000.0,443020000.0


In [7]:
municipalities = df_combined["Municipality"].unique()
for municipality in municipalities:
    display(municipality)

'Aabenraa'

'Aalborg'

'Aarhus'

'Albertslund'

'All Denmark'

...

'Vejle'

'Vesthimmerlands'

'Viborg'

'Vordingborg'

'Ærø'

In [8]:
import pandas as pd

# Load the Excel file using a relative path
df_zipcodes = pd.read_excel('kommuner_zip_codes.xlsx')

# Display the first few rows to verify the data
print(df_zipcodes.head())

# Optional: Check the column names to make sure everything is loaded correctly
print(df_zipcodes.columns)

     Kommune  Postnummer
0  København        1050
1  København        1051
2  København        1052
3  København        1053
4  København        1054
Index(['Kommune', 'Postnummer'], dtype='object')


In [9]:
# Load the zip code dataframe
df_zipcodes = pd.read_excel('kommuner_zip_codes.xlsx')

# Rename the 'Kommune' column in df_zipcodes to match 'Municipality' in df_final_data
df_zipcodes.rename(columns={'Kommune': 'Municipality'}, inplace=True)

# Merge the two dataframes on the 'Municipality' column
df_merged = pd.merge(df_final_data, df_zipcodes, on='Municipality', how='left')

# Save or display the resulting merged dataframe
display(df_merged.head())

# Optional: Save the merged dataframe to a new Excel file if needed
df_merged.to_excel('final_merged_data.xlsx', index=False)


,Municipality,AverageAge,NumberOfChildren,Men,Women,AverageIncome,Elementary school,High school education,Vocational Education,Short Higher Education,...,Longer Higher Education,PhD and research programs,Elementary school expense,Special Education expense,Elderly Care expense,Healthcare expense,Green Spaces expense,Infrastructure expense,Social Services expense,Postnummer
0,Aabenraa,44.442998,11416.0,29422.0,29580.0,337605.0,6870.0,1357.0,1076.0,70.0,...,49.0,8.0,368550000.0,679000.0,322629000.0,1710000.0,28000.0,46206000.0,35892000.0,6200.0
1,Aabenraa,44.442998,11416.0,29422.0,29580.0,337605.0,6870.0,1357.0,1076.0,70.0,...,49.0,8.0,368550000.0,679000.0,322629000.0,1710000.0,28000.0,46206000.0,35892000.0,6210.0
2,Aabenraa,44.442998,11416.0,29422.0,29580.0,337605.0,6870.0,1357.0,1076.0,70.0,...,49.0,8.0,368550000.0,679000.0,322629000.0,1710000.0,28000.0,46206000.0,35892000.0,6230.0
3,Aabenraa,44.442998,11416.0,29422.0,29580.0,337605.0,6870.0,1357.0,1076.0,70.0,...,49.0,8.0,368550000.0,679000.0,322629000.0,1710000.0,28000.0,46206000.0,35892000.0,6330.0
4,Aabenraa,44.442998,11416.0,29422.0,29580.0,337605.0,6870.0,1357.0,1076.0,70.0,...,49.0,8.0,368550000.0,679000.0,322629000.0,1710000.0,28000.0,46206000.0,35892000.0,6340.0


In [10]:
# Omdøb kolonnen fra 'Postnummer' til 'Postal code'
df_merged = df_merged.rename(columns={'Postnummer': 'Postal code'})

# Bekræft ændringen ved at printe kolonneoverskrifterne
print(df_merged.columns)


# Læs den nye fil ind som en Pandas DataFrame
final_data_v1 = pd.read_csv('Processed_Housing_Data_Final.csv')

display(final_data_v1)
# Kontroller, at postnummerkolonnen har samme navn i begge DataFrames
# Hvis ikke, omdøb den passende kolonne
# For eksempel hvis postnummerkolonnen i 'new_data' hedder noget andet, så brug denne linje:
# new_data = new_data.rename(columns={'Postnummer': 'Postnummer2'})

# Merge de to DataFrames på basis af postnummer
merged_data_final = pd.merge(final_data_v1, df_merged, on='Postal code', how='left')
merged_data_final = pd.merge(final_data_v1, df_merged, left_on='Postal code', right_on='Postal code', how='left')

# Vis de første par rækker af det sammenflettede datasæt for at bekræfte korrekt sammenfletning
display(merged_data_final.head())

# Gem det sammenflettede datasæt, hvis det ønskes
merged_data_final.to_csv('final_data_with_housing.csv', index=False)

# Export the entire merged dataset to Excel
merged_data_final.to_excel('final_merged_data_v2.xlsx', index=False)

Index(['Municipality', 'AverageAge', 'NumberOfChildren', 'Men', 'Women',
       'AverageIncome', 'Elementary school', 'High school education',
       'Vocational Education', 'Short Higher Education',
       'Intermediate Higher Education', 'Bachelor's programmes',
       'Longer Higher Education', 'PhD and research programs',
       'Elementary school expense', 'Special Education expense',
       'Elderly Care expense', 'Healthcare expense', 'Green Spaces expense',
       'Infrastructure expense', 'Social Services expense', 'Postal code'],
      dtype='object')


,Adress,City,Street name,Postal code,Latitude,Longitude,Rooms,Property type,Energy label,Price,...,Home size,Monthly expense,Ground size,Transport Station Distance (km),Beach Distance (km),School Distance (km),Restaurant Distance (km),Hospital Distance (km),Mall Distance (km),Supermarkets Distance (km)
0,"Nørregade 27, Agerskov 6534",Agerskov,Nørregade,6534,55.130960,9.136025,4,Villa,E,495000,...,100,1.079,1015,16.937517,33.888472,0.218910,0.420885,16.986235,32.349696,8.100554
1,"Keldsvej 5, Store Fuglede 4480",Store Fuglede,Keldsvej,4480,55.577656,11.180144,7,Villa,C,1695000,...,223,2.079,924,11.508389,28.718417,1.954803,9.338248,22.943350,10.673426,5.429859
2,"Tofteengen 8, Roskilde 4000",Roskilde,Tofteengen,4000,55.693270,12.147028,7,Villa,NaN,6495000,...,190,2.944,834,5.142713,4.930456,1.622775,2.926239,7.211322,6.758098,4.860035
3,"Længstedal 16, Skjern 6900",Skjern,Længstedal,6900,56.003277,8.696856,7,Villa/Landejendom,C,1495000,...,224,2.222,17768,3.319680,14.497442,5.857701,5.974750,20.908849,29.103602,10.622460
4,"Åskrænten 22, Egå 8250",Egå,Åskrænten,8250,56.203278,10.264264,5,Villa,C,6998000,...,163,4.825,810,1.937711,1.270812,1.098538,1.355281,5.404396,3.411354,1.215941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9910,"Bødkerstræde 10, Karlslunde 2690",Karlslunde,Bødkerstræde,2690,55.570120,12.224564,6,Villa,C,5995000,...,190,4.508,1536,4.115616,2.672530,0.262239,1.427502,3.071907,2.343683,0.218098
9911,"Odinsvej 2, Kirke Såby 4060",Kirke Såby,Odinsvej,4060,55.651460,11.880332,3,Villa,C,1645000,...,72,1.780,611,6.387605,3.677477,1.707735,6.612146,12.249469,13.359499,6.426935
9912,"Purupvej 14, Østbirk 8752",Østbirk,Purupvej,8752,55.987175,9.734029,5,Villa,E,2495000,...,233,2.636,2296,11.691203,10.109543,2.951033,0.942083,16.320359,13.302073,9.313335
9913,"Vinkelvej 9, Ulfborg 6990",Ulfborg,Vinkelvej,6990,56.375076,8.121837,1,Villa,D,695000,...,64,865.000,418,13.806256,30.125570,8.263172,0.931642,22.786710,30.593379,14.250801


,Adress,City,Street name,Postal code,Latitude,Longitude,Rooms,Property type,Energy label,Price,...,Bachelor's programmes,Longer Higher Education,PhD and research programs,Elementary school expense,Special Education expense,Elderly Care expense,Healthcare expense,Green Spaces expense,Infrastructure expense,Social Services expense
0,"Nørregade 27, Agerskov 6534",Agerskov,Nørregade,6534,55.130960,9.136025,4,Villa,E,495000,...,12.0,20.0,2.0,2.165870e+08,330000.0,2.159480e+08,7853000.0,0.0,35717000.0,29579000.0
1,"Keldsvej 5, Store Fuglede 4480",Store Fuglede,Keldsvej,4480,55.577656,11.180144,7,Villa,C,1695000,...,63.0,41.0,5.0,2.806860e+08,1666000.0,2.223620e+08,4171000.0,156000.0,20071000.0,46418000.0
2,"Tofteengen 8, Roskilde 4000",Roskilde,Tofteengen,4000,55.693270,12.147028,7,Villa,NaN,6495000,...,992.0,631.0,92.0,6.525210e+08,2574000.0,4.390510e+08,17162000.0,500000.0,94707000.0,54150000.0
3,"Længstedal 16, Skjern 6900",Skjern,Længstedal,6900,56.003277,8.696856,7,Villa/Landejendom,C,1495000,...,16.0,25.0,6.0,3.704340e+08,3000000.0,2.713550e+08,5691000.0,1839000.0,47807000.0,36748000.0
4,"Åskrænten 22, Egå 8250",Egå,Åskrænten,8250,56.203278,10.264264,5,Villa,C,6998000,...,14294.0,10542.0,1470.0,2.226985e+09,4347000.0,1.442222e+09,68896000.0,5633000.0,75722000.0,113920000.0
